<a href="https://colab.research.google.com/github/NelsenYoung/GPT_adventure/blob/main/GPT_adventure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup of the boilerplate
import torch
import torch.nn.functional as F

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-08 21:40:28--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  6.79MB/s    in 0.2s    

2026-01-08 21:40:28 (6.79 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt") as f:
  text = f.read()
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# Encode the dataset
chars = sorted(list(set(text)))
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for i, s in enumerate(chars)}

In [5]:
# Create the character-wise dataset
block_size = 8

X = []
Y = []

# For each token, grab a block
for i in range(len(text) - (block_size)):
  xs = text[i:(i+block_size)]
  ys = text[i+block_size]

  # Encode the block
  xs = [stoi[i] for i in xs]
  ys = [stoi[i] for i in ys]

  X.append(xs)
  Y += ys


In [6]:
print(len(X), len(Y))
num_examples = len(X)

1115386 1115386


In [7]:
X = torch.tensor(X)
Y = torch.tensor(Y)

In [8]:
# Split the data into test and train splits
test_train_split = 8
split = (num_examples // 10) * test_train_split
X_train = X[:split]
Y_train = Y[:split]
X_test = X[split:]
Y_test = Y[split:]

In [9]:
num_classes = len(set(text))
emb_dims = 12
num_hidden = 200

model = torch.nn.Sequential(
    torch.nn.Embedding(num_classes, emb_dims),
    torch.nn.Flatten(1),
    torch.nn.Linear(block_size * emb_dims, num_hidden),
    torch.nn.Tanh(),
    torch.nn.Linear(num_hidden, num_classes)
)

In [38]:
epochs = 100000
lr = 1e-3
momentum = 0.9
batch_size = 32
train_data_size = X_train.shape[0]
optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for i in range(epochs):
  # mini batch
  idxs = torch.randint(train_data_size, (batch_size,))
  x = X_train[idxs]
  y = Y_train[idxs]

  # forward pass
  logits = model.forward(x)

  # backward pass
  optim.zero_grad()
  loss = F.cross_entropy(logits, y)
  loss.backward()
  optim.step()

  if i % 10000 == 0:
    print(f'epoch {i}: {loss}')

with torch.no_grad():
  # Training loss
  x = X_train
  y = Y_train

  # forward pass
  logits = model.forward(x)
  loss = F.cross_entropy(logits, y)
print(f'Training Loss: {loss}')

with torch.no_grad():
  # Test loss
  x = X_test
  y = Y_test

  # forward pass
  logits = model.forward(x)
  loss = F.cross_entropy(logits, y)
print(f'Test Loss: {loss}')

KeyboardInterrupt: 

In [117]:
# Create a way to make predictions
# predictions are made using a forward pass
# we need a way to create an inital 8 chars
window = ["." for _ in range(block_size)]
while True:
  # convert window to a tensor
  x = []
  for c in window:
    x.append(stoi[c])
  x = torch.tensor(x).view(1, -1)

  logits = model(x).view(num_classes)
  probs = F.softmax(logits, 0)
  idx = torch.multinomial(probs, 1).item()

  char = itos[idx]
  print(char, end="")

  window.append(char)
  window = window[1:]



And touth brater this neach of be secuting for Inly would, whore and;
Mo yo fell,
Fors come the bun
Gatovannr'd foo the to;
And preat;
CLHING HOWAowe,
:'s their
Yie faysons Blew
In these un a more, who him; so eany rige
Bo lover sone, I cear not blood:
What revent
They what is noir so dount Io,:
Yot, pert dittech pep'et
Moug' Raysirded you hread freag he why abe
And of Clave my with!'
fore footh arvious; Louth, you, son, these! for maipuy.

KING EV:
Wech anus?

Ke Comes,
Come plowong,
But if thice siny the woilded in comefy herline;
To he pry to life me pos it iclown hordefore; our prisey, and day, my gall yills of Lhe brungs the gieeclo's sobent in bediers be a, heacend tolnd Moug lopens.

LIIDT:
Gois, sey, he lorden: fors agove minder for seen till we you krack her roces;
As tay buchas their; fall them breayon,
And mestry. though, dut rost,
But The her dours wardhrwxch nive anate all

Whicele, when your bered,
Do desoner'd their the he.

TERCA:
Fay, whey un yourse are ppeard ruge a,

KeyboardInterrupt: 

In [ ]:
# NOW USING THE TRANSFORMER ARCHITECTURE

In [39]:
# lets first start by going over the idea of attention
# We take in a BxTxC tensor (lets say 4 x 8 x 10)
# Each token create a Q and K vector and we take the dot product between all of them which should result in a B x T x T
  # The way that is works is that Q and K are B x T x E (embedding size) so Q @ K^T = B x T x T
# The result of this would be considered the Wei matrix and we will turn this into a lower triangular matrix because we only
# want previous tokens to influence future ones
# Then we want to create our value matrix (C x H) and pass the input through it to get v which is B x T x H (the values matrix
# chooses which channels of x to chose from)
# Wei then plucks out the most important values from this value matrix B x T x T @ B x T x H = B x T x H

B, T, C, H = batch_size, block_size, emb_dims, 8
x = torch.randn(B, T, C)

W_Q = torch.randn(C, H)
W_K = torch.randn(C, H)
W_V = torch.randn(C, H)

Q = x @ W_Q
K = x @ W_K
Wei = Q @ K.transpose(1, 2)
Wei = torch.tril(Wei)
Wei = Wei.masked_fill(Wei == 0, float('-inf'))
Wei = F.softmax(Wei, 2)

V = x @ W_V
out = Wei @ V
out.shape

torch.Size([32, 8, 8])

In [40]:
class Head:
  def __init__(self, head_size):
    self.W_Q = torch.randn(emb_dims, head_size)
    self.W_K = torch.randn(emb_dims, head_size)
    self.W_V = torch.randn(emb_dims, head_size)
  def __call__(self, x):
    Q = x @ self.W_Q
    K = x @ self.W_K
    Wei = Q @ K.transpose(1, 2)
    Wei = torch.tril(Wei)
    Wei = Wei.masked_fill(Wei == 0, float('-inf'))
    Wei = F.softmax(Wei, 2)

    V = x @ self.W_V
    return Wei @ V

In [46]:
test = Head(emb_dims)
out = test(x)